In [2]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.6/479.6 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 7.8 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 12.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 10.9 MB/s eta 0:00:0000:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 11.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 9.7 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Define a custom R² metric for Keras as a proxy for "accuracy" in regression.
def r2_metric(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - SS_res/(SS_tot + tf.keras.backend.epsilon())

def preprocess_data(filepath):
    """
    Reads the Census data, drops unwanted weight columns,
    and processes features based on their attribute types.
    """
    # Read data from the Excel file
    df = pd.read_excel(filepath)
    
    # Drop the census weight columns
    drop_cols = ['HSUP_WGT', 'MARSUPWT', 'FSUP_WGT']
    for col in drop_cols:
        if col in df.columns:
            df = df.drop(columns=col)
    
    # Target variable is AGI.
    y = df['AGI']
    
    # Remove target column from features
    X = df.drop(columns=['AGI'])
    
    # Binary and ordinal features are assumed to be already numeric.
    # One-hot encode categorical columns: PAW_YN, A_MARITL, PENATVTY.
    categorical_cols = ['PAW_YN', 'A_MARITL', 'PENATVTY']
    X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
    
    # Ensure all remaining columns are numeric.
    for col in X.columns:
        if X[col].dtype == 'object':
            X[col] = pd.to_numeric(X[col], errors='coerce')
    
    # Handle any missing values by dropping rows (or consider imputation if desired)
    X = X.dropna()
    y = y[X.index]
    
    return X, y

def build_model(input_dim, architecture):
    """
    Constructs a Keras Sequential model with the given hidden layer sizes.
    Each hidden layer uses ReLU activation, and the output layer is linear.
    """
    model = Sequential()
    # First hidden layer with the specified input dimension
    model.add(Dense(architecture[0], activation='relu', input_dim=input_dim))
    # Additional hidden layers as specified by the architecture tuple
    for units in architecture[1:]:
        model.add(Dense(units, activation='relu'))
    # Output layer: one neuron for regression (linear activation)
    model.add(Dense(1, activation='linear'))
    
    # Compile model using the Adam optimizer, Mean Squared Error loss,
    # and track our custom R² metric along with MAE.
    model.compile(optimizer='adam', loss='mse', metrics=[r2_metric, 'mae'])
    return model

def train_and_evaluate_model(X_train, X_test, y_train, y_test, architecture):
    print("---------------------------------------------------")
    print(f"Architecture: {architecture}")
    
    input_dim = X_train.shape[1]
    model = build_model(input_dim, architecture)
    
    # Set up early stopping; here we monitor the validation loss with min_delta of 0.0005.
    early_stop = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0005, restore_best_weights=True)
    
    # Train the model, using 20% of the training data for validation.
    model_history = model.fit(X_train, y_train, 
                        validation_split=0.2, 
                        epochs=200, 
                        callbacks=[early_stop], 
                        verbose=1)
    
    epochs_trained = len(model_history.history['loss'])
    
    # Plot training loss (MSE) and validation "accuracy" (R² metric) versus epoch.
    print(model_history.history.keys())
    plt.figure(figsize=(8,5))
    plt.plot(model_history.history['loss'], label='Training Loss (MSE)')
    plt.plot(model_history.history['val_loss'], label='Validation Loss (MSE)')
    plt.xlabel('Epoch')
    plt.ylabel('Metric Value')
    plt.title(f"Training Curves for Architecture {architecture}")
    plt.legend()
    plt.savefig(f"training_curve_{'_'.join(map(str, architecture))}.png")  # This line saves the figure
    plt.show()
    
    # Evaluate the model on the full training set and test set.
    train_eval = model.evaluate(X_train, y_train, verbose=0)
    test_eval = model.evaluate(X_test, y_test, verbose=0)
    
    # The model.evaluate() returns [loss, r2_metric, mae]
    train_mse = train_eval[0]
    train_r2 = train_eval[1]
    train_mae = train_eval[2]
    
    test_mse = test_eval[0]
    test_r2 = test_eval[1]
    test_mae = test_eval[2]
    
    # Define generalization gap as the difference between training and test R² scores.
    generalization_gap = train_r2 - test_r2
    
    # Print the model training information
    print(f"Number of epochs: {epochs_trained}")
    print("Training Set Metrics:")
    print(f"  Coefficient of Determination (R²): {train_r2:.4f}")
    print(f"  MSE: {train_mse:.4f}")
    print(f"  MAE: {train_mae:.4f}")
    print("Test Set Metrics:")
    print(f"  Coefficient of Determination (R²): {test_r2:.4f}")
    print(f"  MSE: {test_mse:.4f}")
    print(f"  MAE: {test_mae:.4f}")
    print(f"Generalization Gap (Train R² - Test R²): {generalization_gap:.4f}\n")
    
    return {
        'architecture': architecture,
        'epochs': epochs_trained,
        'train_r2': train_r2,
        'train_mse': train_mse,
        'train_mae': train_mae,
        'test_r2': test_r2,
        'test_mse': test_mse,
        'test_mae': test_mae,
        'generalization_gap': generalization_gap
    }

In [ ]:
def main():
    # Step 1: Read in and preprocess the dataset
    X, y = preprocess_data("Census_Supplement_Data.xlsx")
    
    # Step 2: Split the data into training (70%) and test (30%) sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Step 3: Normalize the data using StandardScaler (fit on training set only)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Define the list of neural network architectures to try.
    architectures = [
        (4, 4),
        (10, 6),
        (32, 16),
        (8, 3, 5),
        (12, 9, 10)
    ]
    
    results = []
    # For each architecture, build, train, and evaluate the model.
    for arch in architectures:
        res = train_and_evaluate_model(X_train, X_test, y_train, y_test, arch)
        results.append(res)
    
    # Display a summary table of results
    results_df = pd.DataFrame(results)
    print("Summary of Model Performances:")
    print(results_df)

if __name__ == '__main__':
    main()


---------------------------------------------------
Architecture: (4, 4)
Epoch 1/200
1750/1750 [==============================] - 4s 2ms/step - loss: 9806257152.0000 - r2_metric: -0.2957 - mae: 39237.8750 - val_loss: 9888379904.0000 - val_r2_metric: -0.2701 - val_mae: 38235.3008
Epoch 2/200
1750/1750 [==============================] - 3s 2ms/step - loss: 9333365760.0000 - r2_metric: -0.2068 - mae: 38475.4570 - val_loss: 9080040448.0000 - val_r2_metric: -0.1143 - val_mae: 36958.6719
Epoch 3/200
1750/1750 [==============================] - 3s 2ms/step - loss: 8293796352.0000 - r2_metric: -0.0228 - mae: 36943.5312 - val_loss: 7884699136.0000 - val_r2_metric: 0.0859 - val_mae: 35421.7227
Epoch 4/200
1750/1750 [==============================] - 3s 2ms/step - loss: 7146820608.0000 - r2_metric: 0.1477 - mae: 35878.8438 - val_loss: 6873478144.0000 - val_r2_metric: 0.1960 - val_mae: 35101.4375
Epoch 5/200
1750/1750 [==============================] - 3s 2ms/step - loss: 6384791040.0000 - r2_metr